In [1]:
import os
import io
import re
import json
import time
import uuid
from typing import List, Dict, Any

import streamlit as st
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

os.chdir('../')

In [2]:
os.getcwd()

'/Users/jameslim/Desktop/GITHUB/ai_km_poc/WebApp_ChatBot'

In [3]:
import sqlite3
import sys

con=sqlite3.connect('.chroma/chroma.sqlite3') 
print('tables:', [r[0] for r in con.execute("SELECT name FROM sqlite_master WHERE type='table'")])
con.close()

tables: ['migrations', 'acquire_write', 'collection_metadata', 'segment_metadata', 'tenants', 'databases', 'collections', 'maintenance_log', 'segments', 'embeddings', 'embedding_metadata', 'max_seq_id', 'embedding_fulltext_search', 'embedding_fulltext_search_data', 'embedding_fulltext_search_idx', 'embedding_fulltext_search_content', 'embedding_fulltext_search_docsize', 'embedding_fulltext_search_config', 'embeddings_queue', 'embeddings_queue_config']


In [4]:
RecordId = "7dbe49bf-93f5-4c7a-99e9-b9b6cd6c5d2a"

In [8]:
# inspect_chroma.py
import chromadb, os

VECTOR_DB_PATH = os.environ.get("VECTOR_DB_PATH", ".chroma")
try:
    client = chromadb.PersistentClient(path=VECTOR_DB_PATH)
except TypeError:
    from chromadb.config import Settings
    client = chromadb.Client(Settings(persist_directory=VECTOR_DB_PATH))

# 1) List collections
print("Collections:", [c.name for c in client.list_collections()])

# 2) Open yours
col = client.get_or_create_collection(name="ai4km_assets", metadata={"hnsw:space":"cosine"})

# 3) Count docs
print("Count:", col.count())

# 4) Fetch a few items (ids, metadatas)
batch = col.get(include=["embeddings","metadatas","documents","uris"], limit=10)
for i, _id in enumerate(batch["ids"]):
    print(f"\n[{i}] id={_id}")
    print("   meta:", batch["metadatas"][i])
    # comment next line if your docs are huge
    print("   doc[preview]:", (batch["documents"][i] or "")[:120].replace("\n"," "))

# 5) Fetch a specific RecordId (match it to the one from SharePoint):
one = col.get(ids=[RecordId], include=["metadatas","documents","embeddings"])
print("\n\n\n\nExact lookup:\n\n", one)


Collections: ['ai4km_assets']
Count: 1

[0] id=7dbe49bf-93f5-4c7a-99e9-b9b6cd6c5d2a
   meta: {'ContentOwner': 'nreniers@amgen.com', 'Title': 'Streamlining RMA Mismatch Resolution', 'Site': 'ABR', 'Function': 'Quality'}
   doc[preview]: Title: Reduction of RMA mismatch resolution time   ContentSummary: The project highlights the frequent mismatches betwee




Exact lookup:

 {'ids': ['7dbe49bf-93f5-4c7a-99e9-b9b6cd6c5d2a'], 'embeddings': array([[-0.01374634, -0.00790247, -0.00269297, ...,  0.0059363 ,
         0.01203225,  0.00439446]], shape=(1, 3072)), 'documents': ['Title: Reduction of RMA mismatch resolution time \n\nContentSummary: The project highlights the frequent mismatches between product complaint return units and the expected details recorded in the Return Material Authorization (RMA). These mismatches include issues such as missing RMA references, incorrect quantities, or lot numbers, which prevent the completion of the Post Goods Receipt (PGR) in the ERP system. As a resul

In [6]:
type(one)

dict

In [9]:
one["ids"]

['7dbe49bf-93f5-4c7a-99e9-b9b6cd6c5d2a']

In [10]:
one["documents"]

['Title: Reduction of RMA mismatch resolution time \n\nContentSummary: The project highlights the frequent mismatches between product complaint return units and the expected details recorded in the Return Material Authorization (RMA). These mismatches include issues such as missing RMA references, incorrect quantities, or lot numbers, which prevent the completion of the Post Goods Receipt (PGR) in the ERP system. As a result, these discrepancies lead to increased time spent resolving mismatches, impacting overall efficiency and increasing waste. With approximately 950 RMA mismatches occurring each year, each requiring around 7 minutes to resolve, this totals to about 110 hours of extra work annually 2. \n\nBenefits: The process improvements led to significant efficiency gains and reduced the workload. \n1. The resolution time of RMA mismatch is reduced with 42% \n2. The reduction of emails sent is reduced with 50%. These improvements resulted in a annual time savings of 75 hours. This 

In [11]:
one["embeddings"]

array([[-0.01374634, -0.00790247, -0.00269297, ...,  0.0059363 ,
         0.01203225,  0.00439446]], shape=(1, 3072))